# Исследование влияния размеров обучающей и проверочной выборок на качество обучения

**Создайте модель для распознавания рукописных цифр из набора MNIST (можно воспользоваться ноутбуком 1-го занятия) и проведите ряд тестов:**

Запустите сеть с различными размерами обучающей и проверочной выборок:

1. Размер `test_size=0.1`
2. Размер `test_size=0.5`
3. Размер `test_size=0.9`

Напишите выводы, как изменение `test_size` влияет на точность сети.